In [1]:
import pandas as pd
import numpy as np
from tqdm import *
from pprint import pprint

In [2]:
traces = [
#     ('T1' , '/home/tiratatp/disk1_test.txt'),
    ('T1' , '/home/tiratatp/Repositories/snia_traces/T1/LiveMapsBackEnd/Combined/disk1_filtered.txt'),
    ('T2' , '/home/tiratatp/Repositories/snia_traces/T2/DisplayAdsDataServer/Combined/disk0.txt'),
    ('T3' , '/home/tiratatp/Repositories/snia_traces/T3/DisplayAdsPayload/Combined/disk0.txt'),
    ('T4' , '/home/tiratatp/Repositories/snia_traces/T4/Exchange-Server-Traces/Combined/disk8.txt'),
    ('T5' , '/home/tiratatp/Repositories/snia_traces/T5/MSNStorageCFS/Combined/disk6_filtered.txt'),
    ('T6' , '/home/tiratatp/Repositories/snia_traces/T6/MSNStorageFileServer/Combined/disk5_filtered.txt'),
    ('T7' , '/home/tiratatp/Repositories/snia_traces/T7/BuildServer/Combined/disk0_filtered.txt'),
    ('T8' , '/home/tiratatp/Repositories/snia_traces/T8/DevelopmentToolsRelease/Combined/disk6_filtered.txt'),
    ('T9' , '/home/tiratatp/Repositories/snia_traces/T9/RadiusAuthentication/Combined/disk0_filtered.txt'),
    ('T10', '/home/tiratatp/Repositories/snia_traces/T10/RadiusBackEndSQLServer/Combined/disk4_filtered.txt'),    
]

# bin size in ms; 1hour
bin_size = 3600000

#info about size of preallocated shelters:
sheltersize = 10
shelterrange = 100

In [3]:
#MB in sectors
one_MB = 2 * 1024
x = sheltersize * one_MB
y = shelterrange * one_MB
assert x < y

In [4]:
def return_merged_trace(trace):    
    # read trace into memory
    t = pd.read_csv(trace, delimiter=' ', usecols=[0,2,3,4], \
                    header=None, names=['offset', 'blkno', 'blkcount', 'flag'], \
                    dtype={'offset':np.float_, 'blkno':np.int_, 'blkcount':np.int_, 'flag':np.bool_}, \
                    na_filter=False, engine='c')

    # compare previous IO and tag it if it's sequential
    t['is_seq'] = (t['flag'].shift(1) == t['flag']) & ((t['blkno'] + t['blkcount']).shift(1) == t['blkno'])
    # use cumsum to help group the IO
    t['io_num'] = (~t['is_seq']).astype(int).cumsum()
    # merge sequential IO
    t = t.groupby(['io_num'], sort=False, as_index=False).agg({
            'offset': 'first',
            'blkno': 'min',
            'blkcount': 'sum',
            'flag': 'first',
        }).reset_index().drop(['index', 'io_num'], 1)
    
    return t

In [24]:
g2_result = {}
g3_result = {}

for trace, f in tqdm(traces):
    t = return_merged_trace(f)
    t['tail'] = t['blkno'] + t['blkcount'] # need for several things
    t_smallwrite = t[(t['flag'] == 0) & (t['blkcount'] <= 64)].drop('flag', 1)
    
    # find the number of shelter    
    t['shifted_tail'] = t['tail'] + ((t['blkno'] / y) * x)
    shelter_count = np.floor(t['shifted_tail'].max() / y)    
    # this is the maximum blocks that can be sheltered in this trace
    max_sheltered_block = x * shelter_count
    
    # second graph
    before_merging_sector = t_smallwrite['blkcount'].sum()
    before_merging_io = len(t_smallwrite.index)
    
    # remove exact IO and then sort by blkno and tail
    g2 = t_smallwrite.drop_duplicates(subset=['blkno', 'blkcount'])\
                                    .sort(['blkno', 'blkcount'])\
                                    .copy(deep=True)
    g2['tail'] = g2['blkno'] + g2['blkcount']
    
    # remove overlapping requests
    before_count = None
    after_count = len(g2)    
    while before_count != after_count:
        before_count = after_count 
        # this should remove most of the overlapping IO
        # except the one that e.g
        # first IO  : ===========
        # second IO :    ===
        # third IO  :          ==
        # The above example is why we need a loop.
        g2['is_ovrlp'] = g2['tail'].shift(1) >= g2['blkno']
        g2['io_num'] = (~g2['is_ovrlp']).astype(int).cumsum()   
        g2 = g2.groupby('io_num', sort=False)\
                .agg({
                    'blkno': 'min',            
                    'tail': 'max',
                })
        after_count = len(g2)
    
    after_merging_sector = (g2['tail'] - g2['blkno']).sum()
    after_merging_io = len(g2.index)
    g2_result[trace] = (
        before_merging_sector,
        after_merging_sector,        
        float(before_merging_sector-after_merging_sector)/before_merging_sector*100,
        before_merging_io,
        after_merging_io,
        float(before_merging_io-after_merging_io)/before_merging_io*100,
        )
    
    # third graph
    sheltered_block_per_period = t_smallwrite['blkcount'].groupby(np.floor(t_smallwrite['offset']/bin_size), sort=False)\
                                                            .sum()
    g3 = sheltered_block_per_period.copy(deep=True)
    if g3.sum() <= max_sheltered_block:
        period = -1
    else:
        period = 0
        # each iteration is one hour longer period
        while not (g3 > max_sheltered_block).any():
            period += 1
            g3 += sheltered_block_per_period.shift(period)                 
    g3_result[trace] = (shelter_count, max_sheltered_block, g3.max(), period)
        
# the result
pprint(g2_result)
pprint(g3_result)

|----------| 0/10   0% [elapsed: 00:00 left: ?, ? iters/sec]{}
{}


In [17]:
for t in g2_result:
    print t, g2_result[t][0], g2_result[t][1], g2_result[t][2], g2_result[t][3], g2_result[t][4], g2_result[t][5]

T8 28194859 12880391 54.3165262859 2505447 862873 65.5601176157
T9 12299174 2979913 75.7714379844 803914 43662 94.5688220382
T6 49487472 17279296 65.083494263 2990478 838993 71.9445185686
T7 142494264 116285326 18.3929775587 3822208 905301 76.3147112873
T4 59741837 11970701 79.9626164827 3622643 471013 86.9980839956
T5 518762 423629 18.3384673511 66655 51638 22.5294426525
T2 1747081 186919 89.3010684679 139889 1722 98.7690240119
T3 3761896 407115 89.1779304904 345427 3200 99.0736103431
T1 18127 8551 52.8272742318 1595 303 81.0031347962
T10 2287446 2140539 6.42231554319 127758 27445 78.5179793046
